# Getting started

First, we enable the cluster to scale up. Note that if you run an auto-scaling cluster,
Google will suspend your nodes. Make sure to have the experiment prepared before running the commands.

The following is assumed ready:
* GKE/Kubernetes cluster (see also `terraform/terraform_notebook.ipynb`)
    * 2 nodes pools (default for system & dependencies, experiment pool)
* Docker image (including dataset, to speed-up starting experiments).
    * Within a bash shell
        * Make sure to have the `requirements-cpu.txt` installed (or `requirements-gpu.txt (in a virtual venv/conda environment). You can run `pip3 install -r requirements-cpu.txt`
    * First run the extractor (locally) `python3 -m fltk extractor configs/example_cloud_experiment.json`
        *  This downloads datasets to be included in the docker image.
    * Build the container `DOCKER_BUILDKIT=1 docker build --platform linux/amd64 . --tag gcr.io/$PROJECT_ID/fltk`
    * Push to your gcr.io repository `docker push gcr.io/$PROJECT_ID/fltk`


With that setup, first set some variables used throughout the experiment.


In [14]:
##################
### CHANGE ME! ###
##################
PROJECT_ID="test-bed-fltk-group16-mb"
CLUSTER_NAME="fltk-testbed-cluster"
DEFAULT_POOL="default-node-pool"
EXPERIMENT_POOL="medium-fltk-pool-1"
REGION="us-central1-c"

# In case we do not yet have the credentials/kubeconfig
gcloud container clusters get-credentials $CLUSTER_NAME --region $REGION --project $PROJECT_ID

Fetching cluster endpoint and auth data.
kubeconfig entry generated for fltk-testbed-cluster.


Scale the default-node-pool up.

In [15]:
# These commands might take a while to complete.
gcloud container clusters resize $CLUSTER_NAME --node-pool $DEFAULT_POOL \
     --num-nodes 1 --region $REGION --quiet

Resizing fltk-testbed-cluster...done.


Updated [https://container.googleapis.com/v1/projects/test-bed-fltk-group16-mb/zones/us-central1-c/clusters/fltk-testbed-cluster].


## Preparation
In case you have already tested something or ran another experiment, we have to remove the deployment of the Orchestrator. This will not delete any experiment data, as this persists on one of the ReadWriteMany PVCs.


Currently, the Orchestrator is deployed using a `Deployment` definition, a future version will replace this with a `Deployment` definition, to make this step unnecessary. For experiments this means the following:

1. A single deployment can exist at a single time in a single namespace. This includes 'completed' experiments.
2. For running batches of experiments, a BatchOrchestrator is provided.


ℹ️ This will not remove any data, but if your orchestrator is still/already running experiments, this will stop the deployment. Running training jobs will not be stopped, for this you can use `kubectl`. ConfigMaps created by the Orchestrator (to provide experiment configurations), will not be removed. See the commented code in the cell below.

In [16]:
# If you want to delete all pytorch trainjobs, uncomment the command below.
# kubectl delete pytorchjobs.kubeflow.org --all --namespace test

# If you want to delete all existing configuration map objects in a namespace, run teh command below
# kubectl delete configmaps --all --namespace test

helm uninstall -n test flearner

W1008 15:43:18.015020   19472 gcp.go:120] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.25+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
Error: uninstall: Release not loaded: flearner: release: not found


: 1

## Install extractor

Deploy the TensorBoard service and persistent volumes, required for deployment of the orchestrator's chart.

In [17]:
helm upgrade --install -n test extractor ../charts/extractor -f ../charts/fltk-values.yaml \
    --set provider.projectName=$PROJECT_ID

W1008 15:43:23.456979   19496 gcp.go:120] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.25+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
Release "extractor" does not exist. Installing it now.
NAME: extractor
LAST DEPLOYED: Sat Oct  8 15:43:25 2022
NAMESPACE: test
STATUS: deployed
REVISION: 1
TEST SUITE: None
NOTES:
Get the FLTK extractors Tensorboard URL by running:

export POD_NAME=$(kubectl get pods -n test -l "app.kubernetes.io/name=fltk.extractor" -o jsonpath="{.items[0].metadata.name}")
echo http://localhost:6006/
kubectl -n test port-forward $POD_NAME 6006:6006


## Define experiment configuration files

Deployment of experiments is currently done through a Helm Deployment. A future release (™️) will rework this to a Job definition, as this allows to re-use the template more easily.


> The `EXPERIMENT_FILE` will contain the description of the experiments
> The `CLUSTER_CONFIG` will contain shared configurations for logging, Orchestrator configuration and replication information.

In [18]:
EXPERIMENT_FILE="../configs/federated_tasks/example_arrival_config.json"
CLUSTER_CONFIG="../configs/example_cloud_experiment.json"

## Setup experiment variables
Next, we will deploy the experiments.


We provide a configuration file, `charts/fltk-values.yaml`, in here change the values under the `provider` block. Change `projectName` to your Google Cloud Project ID.

```yaml
provider:
    domain: gcr.io
    projectName: CHANGE_ME!
    imageName: fltk:latest
```

We use the `--set-file` flag for `helm`, as currently, Helm does not support using files outside of the chart root directory (in this case `charts/orchestrator`). Using `--set-file` we can dynamically provide these files. See also issue [here](https://github.com/helm/helm/issues/3276)


In [19]:
helm uninstall -n test experiment-orchestrator
helm install -n test experiment-orchestrator ../charts/orchestrator -f ../charts/fltk-values.yaml \
    --set-file orchestrator.experiment=$EXPERIMENT_FILE,orchestrator.configuration=$CLUSTER_CONFIG \
    --set provider.projectName=$PROJECT_ID

W1008 15:44:53.546784   19627 gcp.go:120] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.25+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
Error: uninstall: Release not loaded: experiment-orchestrator: release: not found
W1008 15:44:54.300315   19641 gcp.go:120] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.25+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
NAME: experiment-orchestrator
LAST DEPLOYED: Sat Oct  8 15:44:56 2022
NAMESPACE: test
STATUS: deployed
REVISION: 1
TEST SUITE: None
NOTES:
You successfully launched an experiment configuration on your cluster in test.

N.B. Make sure to collect all data after completing your experiment!
N.B. Re-installing the orchestrator WILL RESULT IN DELETION OF ALL TRAINJOBS and PODS!


In [29]:
# To get logs from the orchestrator
kubectl logs -n test fl-server

W1008 15:52:44.674206   20316 gcp.go:119] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.26+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
10-08-2022 13:45:07 root         INFO     Loading file config/configuration.fltk.json
No argument path is provided.
10-08-2022 13:45:07 root         INFO     Starting in cluster mode.
10-08-2022 13:45:07 root         INFO     Starting with experiment replication: 0 with seed: 42
10-08-2022 13:45:07 root         INFO     Starting as Orchestrator
10-08-2022 13:45:07 root         INFO     Starting Orchestrator, initializing resources....
10-08-2022 13:45:07 root         INFO     Loading in cluster configuration file
10-08-2022 13:45:07 root         INFO     Pointing configuration to in cluster configuration.
/opt/bitnami/python/lib/python3.8/site-packages/dataclasses_json/core.py:171: RuntimeWarning: `NoneType` object value of non-optional ty

10-08-2022 13:47:58 root         INFO     Waiting for 53fd0e7c-a960-4b2a-b3ea-e3a2b0df918f to complete, 0 pending, 0 arrivals
10-08-2022 13:48:00 ResourceWatchDog INFO     Fetching pod information of cluster...
10-08-2022 13:48:03 root         INFO     Waiting for 53fd0e7c-a960-4b2a-b3ea-e3a2b0df918f to complete, 0 pending, 0 arrivals
10-08-2022 13:48:08 root         INFO     Waiting for 53fd0e7c-a960-4b2a-b3ea-e3a2b0df918f to complete, 0 pending, 0 arrivals
10-08-2022 13:48:10 ResourceWatchDog INFO     Fetching pod information of cluster...
10-08-2022 13:48:11 ResourceWatchDog INFO     Fetching pod information of cluster...
10-08-2022 13:48:13 root         INFO     Waiting for 53fd0e7c-a960-4b2a-b3ea-e3a2b0df918f to complete, 0 pending, 0 arrivals
10-08-2022 13:48:18 root         INFO     Waiting for 53fd0e7c-a960-4b2a-b3ea-e3a2b0df918f to complete, 0 pending, 0 arrivals
10-08-2022 13:48:21 ResourceWatchDog INFO     Fetching pod information of cluster...
10-08-2022 13:48:23 root      

In [30]:
# To get logs from learners (example)
kubectl logs -n test trainjob-53fd0e7c-a960-4b2a-b3ea-e3a2b0df918f-master-0

# To get logs from learners (federated learning)
kubectl logs -n test trainjob-53fd0e7c-a960-4b2a-b3ea-e3a2b0df918f-worker-0

W1008 15:52:55.407732   20331 gcp.go:119] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.26+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
10-08-2022 13:47:07 root         INFO     Loading file experiments/node.config.yaml
10-08-2022 13:47:07 root         INFO     Failed to get distributed config: Expecting value: line 1 column 1 (char 0), path: experiments/node.config.yaml
10-08-2022 13:47:07 root         INFO     Starting with host=None and port=23456 and interface=None
10-08-2022 13:47:17 fltk.core.node INFO     Configure device for CPU
10-08-2022 13:47:17 root         INFO     Getting net: Nets.fashion_mnist_cnn
10-08-2022 13:47:17 fltk.core.node INFO     world size = 2 with rank=0
10-08-2022 13:47:17 fltk.core.node INFO     federator got a ping from new_sender
10-08-2022 13:47:17 fltk.core.node INFO     Got new client registration from client client1
10-08-2022 13:47:17 

[<=========>] Creating data container at logging/cloud_experiment/Dataset.mnist_Nets.fashion_mnist_cnn_53fd0e7c-a960-4b2a-b3ea-e3a2b0df918f_0/client3.csv
[<=========>] Creating data container at logging/cloud_experiment/Dataset.mnist_Nets.fashion_mnist_cnn_53fd0e7c-a960-4b2a-b3ea-e3a2b0df918f_0/client4.csv
Ending program
W1008 15:52:59.599998   20358 gcp.go:119] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.26+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
Defaulted container "pytorch" out of: pytorch, init-pytorch (init)
10-08-2022 13:47:16 root         INFO     Loading file experiments/node.config.yaml
10-08-2022 13:47:16 root         INFO     Failed to get distributed config: Expecting value: line 1 column 1 (char 0), path: experiments/node.config.yaml
10-08-2022 13:47:16 root         INFO     Starting with host=None and port=23456 and interface=None
10-08-2022 13:47:17 f

10-08-2022 13:47:47 fltk.core.node INFO     [client1] [0/1,   820] loss: 0.063
10-08-2022 13:47:47 fltk.core.node INFO     [client1] [0/1,   830] loss: 0.122
10-08-2022 13:47:47 fltk.core.node INFO     [client1] [0/1,   840] loss: 0.109
10-08-2022 13:47:48 fltk.core.node INFO     [client1] [0/1,   850] loss: 0.114
10-08-2022 13:47:48 fltk.core.node INFO     [client1] [0/1,   860] loss: 0.066
10-08-2022 13:47:48 fltk.core.node INFO     [client1] [0/1,   870] loss: 0.036
10-08-2022 13:47:49 fltk.core.node INFO     [client1] [0/1,   880] loss: 0.056
10-08-2022 13:47:49 fltk.core.node INFO     [client1] [0/1,   890] loss: 0.112
10-08-2022 13:47:49 fltk.core.node INFO     [client1] [0/1,   900] loss: 0.040
10-08-2022 13:47:49 fltk.core.node INFO     [client1] [0/1,   910] loss: 0.048
10-08-2022 13:47:50 fltk.core.node INFO     [client1] [0/1,   920] loss: 0.061
10-08-2022 13:47:50 fltk.core.node INFO     [client1] [0/1,   930] loss: 0.071
10-08-2022 13:47:50 fltk.core.node INFO     [RD-0][L

10-08-2022 13:49:18 fltk.core.node INFO     [client1] [0/1,   840] loss: 0.016
10-08-2022 13:49:19 fltk.core.node INFO     [client1] [0/1,   850] loss: 0.034
10-08-2022 13:49:19 fltk.core.node INFO     [client1] [0/1,   860] loss: 0.035
10-08-2022 13:49:19 fltk.core.node INFO     [client1] [0/1,   870] loss: 0.050
10-08-2022 13:49:20 fltk.core.node INFO     [client1] [0/1,   880] loss: 0.038
10-08-2022 13:49:20 fltk.core.node INFO     [client1] [0/1,   890] loss: 0.029
10-08-2022 13:49:20 fltk.core.node INFO     [client1] [0/1,   900] loss: 0.028
10-08-2022 13:49:21 fltk.core.node INFO     [client1] [0/1,   910] loss: 0.032
10-08-2022 13:49:21 fltk.core.node INFO     [client1] [0/1,   920] loss: 0.010
10-08-2022 13:49:21 fltk.core.node INFO     [client1] [0/1,   930] loss: 0.022
10-08-2022 13:49:21 fltk.core.node INFO     [RD-2][LE-0][EE-2] Train duration is 29.19742250442505 seconds
10-08-2022 13:49:23 fltk.core.node INFO     Test duration is 1.6440889835357666 seconds
10-08-2022 13:4

10-08-2022 13:50:50 fltk.core.node INFO     [client1] [0/1,   860] loss: 0.024
10-08-2022 13:50:50 fltk.core.node INFO     [client1] [0/1,   870] loss: 0.026
10-08-2022 13:50:50 fltk.core.node INFO     [client1] [0/1,   880] loss: 0.005
10-08-2022 13:50:51 fltk.core.node INFO     [client1] [0/1,   890] loss: 0.021
10-08-2022 13:50:51 fltk.core.node INFO     [client1] [0/1,   900] loss: 0.021
10-08-2022 13:50:51 fltk.core.node INFO     [client1] [0/1,   910] loss: 0.031
10-08-2022 13:50:52 fltk.core.node INFO     [client1] [0/1,   920] loss: 0.030
10-08-2022 13:50:52 fltk.core.node INFO     [client1] [0/1,   930] loss: 0.013
10-08-2022 13:50:52 fltk.core.node INFO     [RD-4][LE-0][EE-4] Train duration is 28.99895691871643 seconds
10-08-2022 13:50:54 fltk.core.node INFO     Test duration is 1.5928850173950195 seconds
10-08-2022 13:50:58 fltk.core.node INFO     Finished request!
10-08-2022 13:51:09 fltk.core.node INFO     Got call to stop event loop
[W tensorpipe_agent.cpp:726] RPC agent 

## Copy experiment results from the extractor

Extractor holds the experiment results in the format that can be processedby TensorBoard.
In order to download it to the local machine, execute:

In [31]:
EXTRACTOR_POD_NAME=$(kubectl get pods -n test -l "app.kubernetes.io/name=fltk.extractor" -o jsonpath="{.items[0].metadata.name}")

kubectl cp -n test $EXTRACTOR_POD_NAME:/opt/federation-lab/logging ../logging

W1008 15:53:24.312740   20402 gcp.go:119] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.26+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
W1008 15:53:25.573461   20416 gcp.go:119] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.26+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
tar: Removing leading `/' from member names


# Cleanup

## Removing orchestrator

In [32]:
helm uninstall -n test experiment-orchestrator

W1008 15:54:33.962081   20481 gcp.go:120] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.25+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
release "experiment-orchestrator" uninstalled


## Removing extractor

IMPORTANT: Removing extractor chart will result in deleting the already collected experiment results, stored in the NFS!

In [33]:
helm uninstall extractor -n test

W1008 15:55:46.769442   20557 gcp.go:120] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.25+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
release "extractor" uninstalled


## Wrapping up

To scale down the cluster nodepools, run the cell below. This will scale the node pools down and remove all the experiments deployed (on the cluster).

1. Experiments cannot be restarted.
2. Experiment logs will not persist deletion.


In [34]:
# This will remove all information and logs as well.
kubectl delete pytorchjobs.kubeflow.org --all-namespaces --all

gcloud container clusters resize $CLUSTER_NAME --node-pool $DEFAULT_POOL \
    --num-nodes 0 --region $REGION --quiet

gcloud container clusters resize $CLUSTER_NAME --node-pool $EXPERIMENT_POOL \
    --num-nodes 0 --region $REGION --quiet

W1008 15:56:26.492662   20596 gcp.go:119] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.26+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
pytorchjob.kubeflow.org "trainjob-53fd0e7c-a960-4b2a-b3ea-e3a2b0df918f" deleted
Resizing fltk-testbed-cluster...done.                                          
Updated [https://container.googleapis.com/v1/projects/test-bed-fltk-group16-mb/zones/us-central1-c/clusters/fltk-testbed-cluster].
Resizing fltk-testbed-cluster...done.                                          
Updated [https://container.googleapis.com/v1/projects/test-bed-fltk-group16-mb/zones/us-central1-c/clusters/fltk-testbed-cluster].
